## YARA Rule Generation Based on Pattern Engineering

This notebook implements a direct pattern-matching approach to generate YARA rules.
This strategy was chosen over a Machine Learning model due to the small size and imbalanced nature of the provided input files, which made a data-driven model ineffective.

The goal is to parse security indicators (such as file names and hashes) from the input files and convert them into functional YARA rules, demonstrating a practical and robust method for threat signature generation.

In [1]:
import argparse # for command line argument parsing
import csv # for CSV file handling
import os # for operating system interactions
import re # is crucial for the new sanitization step to remove problematic characters.
import traceback # for error handling and debugging
from datetime import datetime # for timestamping the output file

In [2]:
# Check for yara-python availability
try:
    import yara
    YARA_AVAILABLE = True
    print("yara-python available.")
except ImportError:
    YARA_AVAILABLE = False
    print("yara-python not available. Install with pip install yara-python")

yara-python available.


## 1. Data Loading and Preparation

Objective: Load all files from new_input_files and structure them so I can easily process the content.


### Analysis of the Script
This script is designed to robustly load text files from a directory into a Python dictionary. It's written to handle common issues like varied file encodings, ensuring the data is loaded correctly even if the files were saved with different settings.

### How the Code Works
1. Initialization: The script starts by creating an empty dictionary called ```data```. This will store the content of each successfully read file. The keys will be the filenames (without extensions), and the values will be the file contents.

2. Iterating Through Files: It uses ```os.listdir(data_dir)``` to loop through every file and folder in the specified directory. It checks ```os.path.isfile(file_path)``` to make sure it only processes actual files, ignoring any subdirectories.

3. Handling File Encodings: The most critical part of this script is its flexible approach to reading files. Instead of assuming a single encoding, it defines a list of common encodings (```utf-8```, ```utf-16```, ```latin-1```). It then uses a ```try...except``` block to attempt reading the file with each encoding.

* If a ```UnicodeDecodeError``` occurs (meaning the encoding is incorrect), the script catches the error and moves on to the next encoding in the list.

* This approach ensures that the script will find the correct encoding for the file, preventing the program from crashing.

4. Cleaning the Data: Once a file is successfully read, the script performs a small but important cleaning step. It checks if the content starts with the Byte Order Mark (BOM) character (```\ufeff```). This is a special character sometimes used to indicate the encoding of a file. If found, it is removed to ensure the content is clean and doesn't contain hidden characters.

5. Storing and Reporting: Finally, the cleaned content is added to the ```data``` dictionary with the filename as its key. After processing all files, the script prints a summary of how many files were successfully loaded and then returns the ```data``` dictionary for further use.

### Potential Improvements
* Error Reporting: The current script handles errors gracefully by printing a message, but it could be improved by logging failed files to a separate list or file for easier debugging.

* Encoding List: The list of encodings could be expanded to include others, such as cp1252, to increase the chances of successfully reading a wider variety of text files.

* Non-Text Files: The script assumes all files are text-based. For a more general-purpose solution, you could add logic to identify and skip non-text files (e.g., images, PDFs).

In [3]:
# Step 1: Data Loading and Preparation
def load_data_and_handle_encoding(data_dir):
    """
    Loads text content from files in a directory and handles UnicodeDecodeError.

    Args:
        data_dir (str): The path to the directory containing the input files.

    Returns:
        dict: A dictionary with file names as keys and their content as values.
    """
    data = {}

    for filename in os.listdir(data_dir):
        file_path = os.path.join(data_dir, filename)

        if os.path.isfile(file_path):
            label = os.path.splitext(filename)[0]
            print(f"Processing: {filename}")

            # Try different encodings
            encodings = ['utf-8', 'utf-16', 'latin-1']
            content = None
            for encoding in encodings:
                try:
                    with open(file_path, 'r', encoding=encoding) as file_handle:
                        content = file_handle.read()
                    print(f"Read successfully with {encoding}")
                    break
                except UnicodeDecodeError:
                    print(f"Failed to read with {encoding}")
                    continue
                except (OSError, IOError) as encoding_error:
                    print(f"Error reading {filename} with {encoding}: {encoding_error}")
                    content = None
                    break
            if content is not None:
                # Remove BOM if it was read as a character
                if content.startswith('\ufeff'):
                    content = content[1:]

                data[label] = content
                print(f"Loaded content: {len(content)} characters")
            else:
                print("Empty or unreadable file")

    print(f"\n Total files uploaded: {len(data)}")
    return data


## 2. YARA Rule Generation

Objective: Analyze the contents of each file and build the YARA rule syntax.

Considerations:

Rules for .txt: If text files (such as Adware.txt) contain lists of strings or file names, the code must read each line and convert it into a YARA string.

Rules for .csv: The CSV file likely has columns with hashes (MD5, SHA256) or signature names.

### YARA Rule Generator Script

This script is designed to generate YARA rules from different types of text files. YARA is a powerful tool used in cybersecurity to identify malware based on unique patterns. The primary goal of this code is to take a list of strings from a file (like a CSV or a plain text file) and format them into a syntactically correct and functional YARA rule.

### How the Code Works

```_process_csv_data``` Function


* Purpose: This function is specifically for processing data from CSV files.

* Header Detection: It intelligently checks if the first row is a header by looking for keywords like 'name' or 'signature'. This makes the script adaptable to different CSV formats.

* String Limiting: It includes a critical safety mechanism to limit the number of strings to a maximum of 9,000, preventing the generated rule from exceeding YARA's internal limit of 10,000 strings.

* String Formatting: For each line, it sanitizes the string by stripping whitespace and escaping special characters like ```\``` and ```"``` to ensure it's a valid YARA string.

```_process_text_data``` Function

* Purpose: This function handles general text files where each line is a potential YARA string.

* Line Processing: It ignores comment lines that start with ```#``` or ```//```, which allows users to add comments to their files.

* String Truncation: It checks to ensure no string exceeds YARA's maximum length of 8,192 characters. If a string is too long, it's automatically truncated and a warning is printed.

* Practical Limit: It enforces a practical limit of 100 strings per rule to improve readability and prevent the rule from becoming too unwieldy.

```generate_yara_rule_text``` Function

* Purpose: This is the main function that orchestrates the entire process of rule generation.

* Sanitization: It first sanitizes the entire file content by removing non-printable characters.

* Rule Identifier: It creates a clean YARA identifier from the file's label by removing non-alphanumeric characters.

* Type-Based Processing: It determines whether to use the CSV-specific or general text-processing function based on a simple check of the filename (e.g., if the file ends with "signatures_list").

* Rule Generation: Finally, it formats all the processed strings into a complete YARA rule, including the ```strings``` and ```condition``` sections. The condition ```any of them``` means the rule will trigger if any of the defined strings are found.

In essence, this script automates the creation of YARA rules from raw data, including critical error-checking and formatting that are necessary to produce a valid and usable rule.

### Potential Improvements
* Extended Format Support: The script could be extended to handle other formats like JSON or XML, making it more versatile.

* Rule Metadata: You could add functionality to include additional metadata (e.g., author, description, creation date) within the rule, which is a common practice in professional YARA rules.

* Performance Optimization: For extremely large files, the script could be optimized to process data in chunks rather than loading the entire file into memory at once.

In [4]:
# Step 2: YARA Rule Generation
def _process_csv_data(lines, label):  # pylint: disable=unused-argument
    """
    Process CSV data and extract YARA strings.

    Args:
        lines (list): List of lines from CSV content.
        label (str): Label for the rule.

    Returns:
        list: List of YARA string definitions.
    """
    if not lines:
        print("Empty CSV file")
        return []

    csv_reader = csv.reader(lines)
    yara_strings = []

    # Skip header if it exists
    try:
        first_row = next(csv_reader)
        header_words = ['name', 'signature', 'hash', 'file']
        if first_row and any(word in first_row[0].lower() for word in header_words):
            print(f"Header detected and skipped: {first_row[0]}")
        elif first_row and first_row[0].strip():
            # Process the first row if it's not a header
            sanitized_string = first_row[0].strip()
            if sanitized_string:
                escaped_string = sanitized_string.replace('\\', '\\\\').replace('"', '\\"')
                yara_strings.append(f'\t$s0 = "{escaped_string}"')
    except StopIteration:
        print("Empty CSV or no data")
        return []

    string_counter = len(yara_strings)
    max_strings_for_csv = 9000  # Well below YARA's 10,000 limit

    for row in csv_reader:
        if row and row[0]:
            # Stop if we've reached the maximum number of strings
            if string_counter >= max_strings_for_csv:
                warning_msg = f"Warning: Limiting CSV rule to {max_strings_for_csv} strings"
                print(f"{warning_msg} to avoid YARA limits")
                break

            sanitized_string = row[0].strip()
            if sanitized_string:
                escaped_string = sanitized_string.replace('\\', '\\\\').replace('"', '\\"')
                yara_strings.append(f'\t$s{string_counter} = "{escaped_string}"')
                string_counter += 1

    return yara_strings


def _process_text_data(lines, yara_identifier):
    """
    Process general text file data and extract YARA strings.

    Args:
        lines (list): List of lines from text content.
        yara_identifier (str): YARA identifier for the rule.

    Returns:
        list: List of YARA string definitions.
    """
    yara_strings = []
    string_counter = 0
    max_string_length = 8192  # YARA string length limit
    max_strings_per_rule = 100  # Practical limit for rule readability

    for line in lines:
        cleaned_line = line.strip()
        if cleaned_line and not cleaned_line.startswith(('#', '//')):
            # Check if we've reached the maximum number of strings
            if string_counter >= max_strings_per_rule:
                break

            # Truncate very long strings to avoid YARA limits
            if len(cleaned_line) > max_string_length:
                cleaned_line = cleaned_line[:max_string_length]
                original_length = len(line.strip())
                print(f"Warning: String truncated for rule {yara_identifier} "
                      f"(original length: {original_length})")

            escaped_line = cleaned_line.replace('\\', '\\\\').replace('"', '\\"')
            yara_strings.append(f'\t$s{string_counter} = "{escaped_line}"')
            string_counter += 1

    return yara_strings


def generate_yara_rule_text(label, content):
    """
    Generates the text for a YARA rule.

    Args:
        label (str): The name of the rule, derived from the file name.
        content (str): The content of the file to be converted into YARA strings.

    Returns:
        str: A string containing the complete YARA rule syntax.
    """
    print(f"Generating rule for: {label}")

    # Create a valid YARA identifier
    yara_identifier = re.sub(r'[^a-zA-Z0-9_]', '_', label)

    # More aggressive sanitization while preserving newlines
    sanitized_content = re.sub(r'[\x00-\x08\x0B\x0C\x0E-\x1F\x7F-\x9F]', '', content)

    # Using re.split to handle various line endings more robustly
    lines = re.split(r'[\r\n]+', sanitized_content.strip())

    # Process based on file type
    if label.lower().endswith("signatures_list"):  # Adjusted condition for flexibility
        yara_strings = _process_csv_data(lines, label)
    else:
        yara_strings = _process_text_data(lines, yara_identifier)

    print(f"Valid strings found: {len(yara_strings)}")

    if not yara_strings:
        print(f"No valid strings found for {label}")
        return ""

    strings_section = "\n".join(yara_strings)
    rule = f'''rule {yara_identifier} : {label} {{
    strings:
{strings_section}
    condition:
        any of them
}}

'''
    return rule

## 3. Compilation and Storage

Objective: Compile the generated YARA rules and save them in a .yar file.

### Analysis of the YARA Rule Compilation and Storage Script

This script handles the final, crucial steps of the YARA rule creation process: compilation, validation, and saving the generated rules. It takes the raw YARA rule text strings, combines them into a single file, and performs several important checks to ensure the rules are both syntactically correct and properly stored for use.

### How the Code Works

```_validate_and_prepare_rules``` Function

* Purpose: This helper function ensures that only valid, non-empty YARA rules are processed.

* Validation: It iterates through a dictionary of rules and discards any rule entries that are blank or contain only whitespace. This prevents empty rules from being included in the final output, which could lead to errors during compilation or analysis.

```_write_rules_to_file``` Function

* Purpose: This function is responsible for writing the combined YARA rule text to a specified file on the disk.

* Error Handling: It uses a ```try...except``` block to gracefully manage potential file system errors, such as a full disk, insufficient writing permissions, or other input/output issues (```OSError```, ```IOError```, ```PermissionError```). This ensures the script doesn't crash if it encounters a problem while saving the file.

```_validate_yara_syntax``` Function

* Purpose: This is a critical function that validates the generated YARA rules for correctness.

* Syntax Check: It uses the ```yara-python``` library to compile the rules in memory. This in-memory compilation is the most reliable method for checking if the YARA syntax is valid.

* Compiled Output: If the compilation is successful, it not only confirms the rules are valid but also saves a compiled binary version of the rules (with a ```_compiled.yar``` extension). This compiled format loads significantly faster than plain text rules, offering a major performance benefit for YARA scanners.

* Error Reporting: If a ```yara.SyntaxError``` is detected, it provides a clear, specific error message to the user, indicating where the syntax issue lies. Other compilation errors are also caught and reported.

```_print_file_sample``` Function

* Purpose: This utility function provides a quick visual verification of the generated file's content.

* Preview: It reads and prints the first 15 lines of the output file. This allows for a rapid check to ensure the file was created as expected and contains the correct rule structure.

```compile_and_save_yara_rules``` Function

* Purpose: This is the main orchestrator function that manages the entire compilation and saving process.

* Directory Management: It first ensures that the specified ```output_directory``` exists, creating it if necessary.

* Workflow Orchestration: It calls the various helper functions in a logical sequence: first validating the rules, then writing them to the file, and finally performing the YARA syntax validation and compilation.

* Comprehensive Feedback: It provides extensive print statements throughout the process, giving real-time updates and a summary of the operation. This includes information about the number of valid rules, the total size of the combined rules, the destination file path, and confirmation of successful file creation, along with its size and creation timestamp.

* Global Error Catching: It includes a broad ```try...except``` block to catch any unexpected errors that might occur during the overall process, ensuring a robust and informative execution.

In essence, this script automates the final, critical steps of a YARA rule development workflow, focusing on robust validation, efficient error handling, and performance optimization to deliver reliable and ready-to-use YARA rule sets.

### Potential Improvements
* Detailed Error Logging: While the script prints errors, integrating a more structured logging system (e.g., Python's ```logging``` module) would provide better traceability for debugging in complex environments.

* Rule Set Management: For very large sets of rules, you might consider breaking them into multiple smaller files or organizing them into modules (a YARA feature) to improve manageability and compilation speed.

* Automated Testing: Implementing automated tests for the generated YARA rules (e.g., by scanning known benign and malicious samples) would add another layer of validation and confidence in their effectiveness.

In [5]:
# Step 3: Compilation and Storage
def _validate_and_prepare_rules(rules_dict):
    """
    Validate rules dictionary and prepare valid rules list.

    Args:
        rules_dict (dict): Dictionary of rule labels and rule text.

    Returns:
        list: List of valid rules, empty if none found.
    """
    valid_rules = []
    for label, rule_text in rules_dict.items():
        if rule_text and rule_text.strip():
            valid_rules.append(rule_text)
            print(f"Valid rule included: {label}")
        else:
            print(f"Empty rule omitted: {label}")
    return valid_rules


def _write_rules_to_file(combined_rules, output_file):
    """
    Write combined rules to output file.

    Args:
        combined_rules (str): Combined YARA rules text.
        output_file (str): Path to output file.

    Returns:
        bool: True if successful, False otherwise.
    """
    try:
        with open(output_file, 'w', encoding='utf-8') as output_handle:
            output_handle.write(combined_rules)
        return True
    except (OSError, IOError, PermissionError) as error:
        print(f"Error writing file: {error}")
        return False


def _validate_yara_syntax(combined_rules, output_file):
    """
    Validate YARA syntax and save compiled version if available.

    Args:
        combined_rules (str): Combined YARA rules text.
        output_file (str): Path to output file.
    """
    if not YARA_AVAILABLE:
        print("yara-python not available - Only text file was saved")
        return

    print("\nValidating YARA syntax...")
    try:
        compiled_rules = yara.compile(source=combined_rules)
        print("Valid YARA syntax - Rules compiled correctly")

        compiled_file = output_file.replace('.yar', '_compiled.yar')
        compiled_rules.save(compiled_file)
        print(f"Compiled version saved: {compiled_file}")

    except yara.SyntaxError as syntax_error:
        print(f"YARA syntax error: {syntax_error}")
        print("Text file saved for manual review")
    except (yara.Error, OSError, IOError) as compilation_error:
        print(f"Compilation error: {compilation_error}")
        print("Text file saved correctly")


def _print_file_sample(output_file):
    """
    Print a sample of the generated file.

    Args:
        output_file (str): Path to output file.
    """
    print("\nSample of the generated file:")
    try:
        with open(output_file, 'r', encoding='utf-8') as sample_file:
            lines = sample_file.readlines()[:15]
            for i, line in enumerate(lines, 1):
                print(f"   {i:2d}: {line.rstrip()}")
            if len(lines) >= 15:
                print("   ...")
    except (OSError, IOError) as error:
        print(f"Error reading file sample: {error}")


def compile_and_save_yara_rules(rules_dict, output_directory="../output",
                               filename="compiled_rules.yar"):
    """
    Compiles and saves YARA rules.

    Args:
        rules_dict (dict): A dictionary with YARA rule syntax strings.
        output_directory (str): The path to the output directory.
        filename (str): The name of the output file.

    Returns:
        bool: True if the process was successful, False otherwise.
    """
    print(f"\n{'='*60}")
    print("COMPILING AND SAVING YARA RULES")
    print(f"{'='*60}")

    if not rules_dict:
        print("ERROR: No rules to compile")
        return False

    # Create output directory if needed
    if not os.path.exists(output_directory):
        os.makedirs(output_directory)
        print(f"Directory created: {output_directory}")

    # Validate and prepare rules
    valid_rules = _validate_and_prepare_rules(rules_dict)
    if not valid_rules:
        print("ERROR: No valid rules to save")
        return False

    # Prepare combined rules and output file path
    combined_rules = "\n".join(valid_rules)
    combined_rules = re.sub(r'[\x00]', '', combined_rules)
    output_file = os.path.join(output_directory, filename)

    # Print summary
    print("\nSUMMARY:")
    print(f"   Valid rules: {len(valid_rules)}")
    print(f"   Total size: {len(combined_rules):,} characters")
    print(f"   Destination file: {output_file}")

    try:
        # Write rules to file
        if not _write_rules_to_file(combined_rules, output_file):
            return False

        # Verify file was created
        if not os.path.exists(output_file):
            print("ERROR: The file was not created")
            return False

        # Print success information
        file_size = os.path.getsize(output_file)
        print("\nFILE SAVED SUCCESSFULLY!")
        print(f"   Location: {os.path.abspath(output_file)}")
        print(f"   Size: {file_size:,} bytes")
        print(f"   Created: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

        # Validate YARA syntax and create compiled version
        _validate_yara_syntax(combined_rules, output_file)

        # Print file sample
        _print_file_sample(output_file)

        return True

    except (OSError, IOError, PermissionError) as general_error:
        print(f"UNEXPECTED ERROR: {general_error}")
        traceback.print_exc()
        return False

## 4. Main function to execute all 

Objective: The main goal of this step is to create a complete, functional script that can be executed from beginning to end. It automates the entire YARA rule generation process and provides the capability to test the rules immediately.

### Analysis of the Main Execution Script

This script serves as the orchestrator for the entire YARA rule generation pipeline. Instead of introducing new core functionality, it integrates the previously defined functions for loading data, generating rules, and compiling/saving them into a single, cohesive workflow. This makes the entire process automated and easy to run from the command line.

### How the Code Works

```main_execution_with_output``` Function

* Purpose: This is the control center of the script. It manages the entire process from start to finish.

* Workflow Orchestration: It calls the functions from the three previous steps in the correct order:

  1. ```load_data_and_handle_encoding```: Loads all input files and handles any encoding issues.

  2. ```generate_yara_rule_text```: Converts the loaded file contents into YARA rule syntax.

  3. ```compile_and_save_yara_rules```: Combines, validates, and saves the final rules to a file.

* Error Control: It includes checks after each major step. If no data is loaded or no valid rules are generated, the script terminates early to prevent subsequent errors, providing clear messages to the user.

* User Feedback: It prints status updates at each stage, giving a clear, step-by-step view of the process and its progress.

```scan_file_with_yara``` Function

* Purpose: This function demonstrates how to use the generated YARA rules to scan a file.

* Prerequisites: It first checks if the ```yara-python``` library is installed and if the rule and target files exist, preventing the script from failing on missing dependencies or files.

* Scanning Logic: It uses ```yara.compile(filepath=rules_path)``` to load the rules and then ```compiled_rules.match(filepath=target_file)``` to perform the scan. This is a critical step for verifying the effectiveness of the generated rules.

* Match Reporting: If a match is found, it provides detailed information about which rule was triggered, its associated tags, and the specific strings that matched, which is invaluable for security analysts.

```if __name__ == "__main__"```: Block

* Purpose: This standard Python construct defines the script's entry point when it's run directly from the command line.

* Command-Line Interface: It uses the ```argparse``` module to create a simple but effective command-line interface. This allows users to specify the input and output directories as arguments, making the script highly flexible and suitable for automation or scripting.

* Execution: It takes the command-line arguments and passes them to the ```main_execution_with_output``` function, kicking off the entire process.

In summary, this step combines all the individual components into a complete, executable program. It handles workflow management, argument parsing, and provides a practical demonstration of the tool's final purpose, creating a robust and user-friendly solution.

### Potential Improvements

* Batch Scanning: The ```scan_file_with_yara``` function currently scans only one file at a time. It could be expanded to accept a directory and perform a batch scan of multiple files.

* Configuration File: For more complex use cases, a configuration file (e.g., JSON or YAML) could be used to manage all paths and settings, making the script easier to configure without changing command-line arguments.

* Error Logging: While the script prints errors, implementing a proper logging system would allow for more detailed and structured error reporting, which is crucial for long-running processes or when troubleshooting issues.

In [6]:
# Step 4: Main execution functions
def main_execution_with_output(input_directory, output_directory):
    """
    Main function that executes the entire process with specified output directory.

    Args:
        input_directory (str): Path to the directory with input files.
        output_directory (str): Path to the output directory for compiled rules.
    """
    print("STARTING FULL YARA GENERATION PROCESS")
    print("="*60)

    # 1. Load data
    print("\n1. LOADING DATA...")
    raw_data = load_data_and_handle_encoding(input_directory)

    if not raw_data:
        print("No data loaded. Process terminated.")
        return False

    print(f"Files loaded: {list(raw_data.keys())}")

    # 2. Generate YARA rules
    print("\n2. GENERATING YARA RULES...")
    yara_rules_dict = {}

    for label, content in raw_data.items():
        rule_text = generate_yara_rule_text(label, content)
        if rule_text:
            yara_rules_dict[label] = rule_text

    print(f"\nRules generated: {len(yara_rules_dict)}/{len(raw_data)}")

    if not yara_rules_dict:
        print("No valid rules generated. Process terminated.")
        return False

    # 3. Compile and save
    print("\n3. COMPILING AND SAVING...")
    success = compile_and_save_yara_rules(yara_rules_dict, output_directory)

    if success:
        print("\nPROCESS COMPLETED SUCCESSFULLY!")
    else:
        print("\nError in the compilation process")

    return success

def scan_file_with_yara(rules_path, target_file):
    """
    Scans a target file using a compiled YARA rule set.

    Args:
        rules_path (str): The file path to the compiled YARA rules (.yar).
        target_file (str): The path to the file to be scanned.
    """
    if not YARA_AVAILABLE:
        print("\nERROR: yara-python is not installed. Cannot perform scan.")
        return

    print(f"\n{'='*60}")
    print("🔬 PERFORMING SCAN WITH YARA")
    print(f"{'='*60}")

    if not os.path.exists(rules_path):
        print(f"ERROR: Compiled rules file not found at: {rules_path}")
        return

    if not os.path.exists(target_file):
        print(f"ERROR: Target file not found at: {target_file}")
        return

    try:
        compiled_rules = yara.compile(filepath=rules_path)
        matches = compiled_rules.match(filepath=target_file)

        if matches:
            print(f"MATCH FOUND in {target_file}!")
            for match in matches:
                print(f"   - Rule Name: {match.rule}")
                print(f"     Tags: {', '.join(match.tags)}")
                print(f"     Meta: {match.meta}")
                print("     Strings found:")
                for string_match in match.strings:
                    print(f"       - Offset: {string_match[0]}, "
                          f"Identifier: {string_match[1]}, Data: {string_match[2]}")
        else:
            print(f"No matches found in {target_file}.")

    except yara.Error as yara_error:
        print(f"YARA error during scan: {yara_error}")
    except (OSError, IOError, PermissionError) as general_error:
        print(f"An unexpected error occurred: {general_error}")


if __name__ == "__main__":
    parser = argparse.ArgumentParser(description="Generate YARA rules from input files.")
    parser.add_argument("input_path", type=str,
                        help="Path to the directory with the input files.")
    parser.add_argument("output_path", type=str,
                        help="Path to the output directory for the compiled rules.")

    args = parser.parse_args()

    input_dir = args.input_path
    output_dir = args.output_path

    if not os.path.exists(input_dir):
        print(f"ERROR: Input directory not found: {input_dir}")
    else:
        main_execution_with_output(input_dir, output_dir)


usage: ipykernel_launcher.py [-h] input_path output_path
ipykernel_launcher.py: error: the following arguments are required: input_path, output_path


SystemExit: 2

/Users/dianaterraza/Desktop/data_scientist_yara_project/venv/lib/python3.12/site-packages/IPython/core/interactiveshell.py:3707: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
